In [10]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

import os

from PIL import Image

In [11]:
def load_data():
        data = []
        paths = []
        for r, d, f in os.walk(r"D:\Downloads\selfie2anime\trainB"):
            for file in f:
                if '.jpg' in file:
                    paths.append(os.path.join(r, file))

        for path in paths:
            img = Image.open(path)
            x = img.resize((128,128))
            if(np.array(x).shape == (128,128,3)):
                data.append(np.array(x))

        x_train = np.array(data)
        x_train = x_train.reshape(len(data),128,128,3)
        return x_train

In [12]:
class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 200

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()
        generator = self.generator

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = load_data()

        # Rescale -1 to 1
        X_train = X_train / 256

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 3, 3
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)
        gen_imgs = np.array(gen_imgs) * 256
        gen_imgs = gen_imgs.astype(int)

        fig=plt.figure(figsize=(20, 20))
        for i in range(1, c*r+1):
            img = gen_imgs[i-1]
            fig.add_subplot(r, c, i)
            plt.imshow(img)
        fig.savefig(r"C:\Users\Vee\Desktop\python\GAN\epoch_%d.png" % epoch)
        plt.close()
        self.generator.save("generator.h5")

In [13]:
dcgan = DCGAN()
dcgan.train(epochs=15001, batch_size=40, save_interval=100)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 33, 33, 64)        0         
_________________________________________________________________
batch_normalization_15 (Batc (None, 33, 33, 64)        256       
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 33, 33, 64)       

C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 3.000790, acc.: 21.25%] [G loss: 0.197004]


C:\Users\Vee\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.764576, acc.: 56.25%] [G loss: 2.607033]
2 [D loss: 1.136736, acc.: 46.25%] [G loss: 1.445218]
3 [D loss: 0.731840, acc.: 56.25%] [G loss: 1.102432]
4 [D loss: 0.291231, acc.: 92.50%] [G loss: 1.293458]
5 [D loss: 0.437489, acc.: 78.75%] [G loss: 0.821517]
6 [D loss: 0.215677, acc.: 95.00%] [G loss: 0.536934]
7 [D loss: 0.213768, acc.: 92.50%] [G loss: 0.328721]
8 [D loss: 0.114233, acc.: 98.75%] [G loss: 0.276075]
9 [D loss: 0.112657, acc.: 98.75%] [G loss: 0.352529]
10 [D loss: 0.188565, acc.: 93.75%] [G loss: 0.636465]
11 [D loss: 0.164555, acc.: 95.00%] [G loss: 0.615992]
12 [D loss: 0.440673, acc.: 77.50%] [G loss: 1.013467]
13 [D loss: 0.754302, acc.: 66.25%] [G loss: 1.241324]
14 [D loss: 0.386193, acc.: 85.00%] [G loss: 1.784705]
15 [D loss: 0.468086, acc.: 73.75%] [G loss: 2.801009]
16 [D loss: 0.915320, acc.: 53.75%] [G loss: 2.759919]
17 [D loss: 0.534634, acc.: 80.00%] [G loss: 2.836911]
18 [D loss: 0.863078, acc.: 67.50%] [G loss: 4.254601]
19 [D loss: 1.19249

...

14800 [D loss: 0.594608, acc.: 72.50%] [G loss: 5.812470]
14801 [D loss: 0.368810, acc.: 83.75%] [G loss: 5.251359]
14802 [D loss: 0.290968, acc.: 83.75%] [G loss: 5.715096]
14803 [D loss: 0.324184, acc.: 91.25%] [G loss: 5.386664]
14804 [D loss: 0.175094, acc.: 95.00%] [G loss: 5.695776]
14805 [D loss: 0.290146, acc.: 87.50%] [G loss: 4.412452]
14806 [D loss: 0.292627, acc.: 81.25%] [G loss: 4.707672]
14807 [D loss: 0.175968, acc.: 91.25%] [G loss: 4.692935]
14808 [D loss: 0.424198, acc.: 80.00%] [G loss: 4.413083]
14809 [D loss: 0.220992, acc.: 93.75%] [G loss: 4.912011]
14810 [D loss: 0.253963, acc.: 87.50%] [G loss: 6.107139]
14811 [D loss: 0.280335, acc.: 88.75%] [G loss: 4.896358]
14812 [D loss: 0.356531, acc.: 85.00%] [G loss: 4.802884]
14813 [D loss: 0.562989, acc.: 73.75%] [G loss: 4.491139]
14814 [D loss: 0.106788, acc.: 97.50%] [G loss: 4.941031]
14815 [D loss: 0.221170, acc.: 95.00%] [G loss: 4.750230]
14816 [D loss: 0.361505, acc.: 85.00%] [G loss: 6.685416]
14817 [D loss:

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


14900 [D loss: 0.393563, acc.: 78.75%] [G loss: 4.489954]
14901 [D loss: 0.213640, acc.: 90.00%] [G loss: 6.012242]
14902 [D loss: 0.438635, acc.: 81.25%] [G loss: 3.778145]
14903 [D loss: 0.298389, acc.: 90.00%] [G loss: 5.694045]
14904 [D loss: 0.241313, acc.: 87.50%] [G loss: 4.872241]
14905 [D loss: 0.270813, acc.: 88.75%] [G loss: 5.514491]
14906 [D loss: 0.343897, acc.: 86.25%] [G loss: 6.823974]
14907 [D loss: 0.336265, acc.: 85.00%] [G loss: 4.127679]
14908 [D loss: 0.262084, acc.: 87.50%] [G loss: 5.097282]
14909 [D loss: 0.170782, acc.: 92.50%] [G loss: 6.392472]
14910 [D loss: 0.691158, acc.: 72.50%] [G loss: 4.360838]
14911 [D loss: 0.282540, acc.: 90.00%] [G loss: 6.024563]
14912 [D loss: 0.410169, acc.: 85.00%] [G loss: 4.779184]
14913 [D loss: 0.267164, acc.: 90.00%] [G loss: 4.769074]
14914 [D loss: 0.737976, acc.: 63.75%] [G loss: 6.345975]
14915 [D loss: 0.232301, acc.: 88.75%] [G loss: 6.876376]
14916 [D loss: 0.382811, acc.: 81.25%] [G loss: 3.455814]
14917 [D loss:

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


15000 [D loss: 0.210226, acc.: 87.50%] [G loss: 5.308440]
